In [1]:
import sys
sys.path.append('..')
import os
os.getcwd()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import numpy as np
# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [9]:
Products = pd.read_csv('../Drugs@FDA/' + "Products.txt", sep='\t', on_bad_lines='skip', low_memory=False)
Products

,ApplNo,ProductNo,Form,Strength,ReferenceDrug,DrugName,ActiveIngredient,ReferenceStandard
0,4,4,SOLUTION/DROPS;OPHTHALMIC,1%,0,PAREDRINE,HYDROXYAMPHETAMINE HYDROBROMIDE,0.0
1,159,1,TABLET;ORAL,500MG,0,SULFAPYRIDINE,SULFAPYRIDINE,0.0
2,552,1,INJECTABLE;INJECTION,"20,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
3,552,2,INJECTABLE;INJECTION,"40,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
4,552,3,INJECTABLE;INJECTION,"5,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
...,...,...,...,...,...,...,...,...
48176,761382,1,INJECTABLE;INJECTION,2MG(0.05ML of 40MG/ML),0,ENZEEVU,AFLIBERCEPT-ABZV,0.0
48177,761388,1,INJECTABLE;INTRAVENOUS,340MG/2ML(170MG/ML),0,PIASKY,CROVALIMAB-AKKZ,0.0
48178,761390,1,INJECTABLE;INJECTION,30MG,0,NEMLUVIO,NEMOLIZUMAB-ILTO,0.0
48179,761411,1,INJECTABLE;INTRAVENOUS,50 MG/ML,0,NIKTIMVO,axatilimab-csfr,0.0


In [ ]:
Products_df = pd.read_csv('../Drugs@FDA/' + "Products.txt", sep='\t', on_bad_lines='skip', low_memory=False)
Products_filted = = search(Products_df, drug_input)

In [10]:
folder=""

import os,re,sys,fileinput

for parent,dirnames,filenames in os.walk(folder):
    for filename in filenames:
        if filename.endswith("htm"):
            # print(filename)
            for x in fileinput.input(os.path.join(parent,filename),inplace=1):
                # print(x)
                if re.search(r"<a href=\"http://www.accessdata.fda.gov/drugsatfda_docs\S+.pdf\">",x):
                    # print(re.search(r"<a href=\"http://www.accessdata.fda.gov/drugsatfda_docs\S+.pdf\">",x).group(0))
                    with open("download.list",'a')as f2:
                        f2.write(re.search(r"<a href=\"http://www.accessdata.fda.gov/drugsatfda_docs\S+.pdf\">",x).group(0).replace("<a href=\"","").replace("\">","")+"\n")
                        print(re.sub(r"http://www.accessdata.fda.gov/drugsatfda_docs/label/\d+/","",x))
                else:
                    print(x)

In [13]:
import pandas as pd

In [11]:
import requests
import pandas as pd
from datetime import datetime
import json

def fetch_drug_approvals(start_date=None, end_date=None, limit=100):
    """
    Fetch drug approval data from FDA API
    
    Parameters:
    start_date (str): Start date in format 'YYYY-MM-DD' (optional)
    end_date (str): End date in format 'YYYY-MM-DD' (optional)
    limit (int): Maximum number of records to retrieve
    
    Returns:
    pandas.DataFrame: Processed drug approval data
    """
    
    # FDA API endpoint
    base_url = "https://api.fda.gov/drug/drugsfda.json"
    
    # Build query parameters
    params = {
        "limit": limit,
        "search": "products.marketing_status:\"Prescription\"" # Filter for prescription drugs
    }
    
    # Add date range if specified
    if start_date and end_date:
        date_query = f"submissions.submission_status_date:[{start_date}+TO+{end_date}]"
        params["search"] = params["search"] + " AND " + date_query
    
    try:
        # Make API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Process results
        processed_data = []
        for result in data['results']:
            applications = result.get('submissions', [])
            
            for app in applications:
                approval_info = {
                    'application_number': result.get('application_number'),
                    'sponsor_name': result.get('sponsor_name'),
                    'submission_type': app.get('submission_type'),
                    'submission_status': app.get('submission_status'),
                    'submission_date': app.get('submission_status_date'),
                    'review_priority': app.get('review_priority'),
                }
                
                # Get product details
                products = result.get('products', [])
                if products:
                    approval_info.update({
                        'product_name': products[0].get('brand_name'),
                        'generic_name': products[0].get('active_ingredients', [{}])[0].get('name'),
                        'dosage_form': products[0].get('dosage_form'),
                        'route': products[0].get('route'),
                        'marketing_status': products[0].get('marketing_status')
                    })
                
                processed_data.append(approval_info)
        
        # Convert to DataFrame
        df = pd.DataFrame(processed_data)
        
        # Convert dates to datetime
        if 'submission_date' in df.columns:
            df['submission_date'] = pd.to_datetime(df['submission_date'])
            
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def save_approvals(df, output_file='drug_approvals.csv'):
    """
    Save drug approval data to file
    
    Parameters:
    df (pandas.DataFrame): Drug approval data
    output_file (str): Output filename
    """
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

def analyze_approvals(df):
    """
    Generate basic analysis of drug approval data
    
    Parameters:
    df (pandas.DataFrame): Drug approval data
    
    Returns:
    dict: Analysis results
    """
    analysis = {
        'total_applications': len(df),
        'unique_sponsors': df['sponsor_name'].nunique(),
        'submission_types': df['submission_type'].value_counts().to_dict(),
        'approval_status': df['submission_status'].value_counts().to_dict(),
    }
    
    if 'submission_date' in df.columns:
        analysis['submissions_by_year'] = df['submission_date'].dt.year.value_counts().sort_index().to_dict()
    
    return analysis

In [14]:
# Fetch recent drug approvals (last 6 months)
from datetime import datetime, timedelta
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%d')

# Get the data
approvals_df = fetch_drug_approvals(start_date=start_date, end_date=end_date, limit=100)

# Save to file
save_approvals(approvals_df, 'recent_drug_approvals.csv')

# Analyze the data
analysis = analyze_approvals(approvals_df)
print(json.dumps(analysis, indent=2))

Error fetching data: 500 Server Error: Internal Server Error for url: https://api.fda.gov/drug/drugsfda.json?limit=100&search=products.marketing_status%3A%22Prescription%22+AND+submissions.submission_status_date%3A%5B2024-05-22%2BTO%2B2024-11-18%5D


AttributeError: 'NoneType' object has no attribute 'to_csv'

In [15]:
import requests
import json

# Function to fetch drug approval data from the FDA API
def get_drug_approval(drug_name):
    url = f"https://api.fda.gov/drug/label.json"
    params = {
        'search': f'openfda.brand_name:{drug_name}',  # Searching by brand name
        'limit': 5  # Limits the number of results returned
    }
    
    # Make the GET request to the API
    response = requests.get(url, params=params)
    
    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON data
        
        if 'results' in data:
            for result in data['results']:
                print(f"Drug Name: {result.get('openfda', {}).get('brand_name', 'N/A')}")
                print(f"Manufacturer: {result.get('openfda', {}).get('manufacturer_name', 'N/A')}")
                print(f"Approval Date: {result.get('effective_time', 'N/A')}")
                print(f"Drug Type: {result.get('openfda', {}).get('drug_class', 'N/A')}")
                print("="*50)
        else:
            print(f"No results found for {drug_name}.")
    else:
        print(f"Error: Unable to fetch data (Status Code: {response.status_code})")

# Example usage
drug_name = "Aspirin"  # Replace with the drug you are interested in
get_drug_approval(drug_name)


Drug Name: ['Low Dose Aspirin Enteric Safety-Coated']
Manufacturer: ['P & L Development, LLC']
Approval Date: 20240416
Drug Type: N/A
Drug Name: ['Low Dose Aspirin']
Manufacturer: ['P & L Development, LLC']
Approval Date: 20240416
Drug Type: N/A
Drug Name: ['Rapidol Aspirin']
Manufacturer: ['Pharmadel LLC']
Approval Date: 20230717
Drug Type: N/A
Drug Name: ['Aspirin Low Dose Safety Coated']
Manufacturer: ['P & L Development, LLC']
Approval Date: 20230511
Drug Type: N/A
Drug Name: ['Enteric Coated Aspirin']
Manufacturer: ['DISCOUNT DRUG MART']
Approval Date: 20241016
Drug Type: N/A


In [16]:
drug_name = str(input("Enter drug name: ")) # PENTAVITIN
get_drug_approval(drug_name)

Enter drug name:  Adalimumab


Drug Name: ['Adalimumab']
Manufacturer: ['Golden State Medical Supply, Inc.']
Approval Date: 20231023
Drug Type: N/A
Drug Name: ['Adalimumab-aacf']
Manufacturer: ['Fresenius Kabi USA, LLC']
Approval Date: 20240801
Drug Type: N/A
Drug Name: ['Adalimumab']
Manufacturer: ['Biocon Biologics Inc']
Approval Date: 20240525
Drug Type: N/A
Drug Name: ['Adalimumab-adbm']
Manufacturer: ['Boehringer Ingelheim Pharmaceuticals, Inc.']
Approval Date: 20240502
Drug Type: N/A
Drug Name: ['Adalimumab']
Manufacturer: ['CELLTRION USA, Inc.']
Approval Date: 20231208
Drug Type: N/A


## How do i create a code extract for FDA drug approval?

In [17]:
import requests
import time
import csv

# Constants
BASE_URL = "https://api.fda.gov/drug/label.json"
PAGE_LIMIT = 1000  # Maximum number of results per request
DELAY_BETWEEN_REQUESTS = 1  # Delay in seconds between requests to avoid rate limits

# Function to fetch drug approval data from the FDA API
def fetch_drug_approvals(page=1):
    params = {
        'limit': PAGE_LIMIT,
        'skip': (page - 1) * PAGE_LIMIT  # Pagination: skip previous pages
    }

    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data.get('results', [])
    else:
        print(f"Error fetching data for page {page}: {response.status_code}")
        return []

# Function to process and save drug approvals data to CSV
def save_drug_approvals_to_csv(filename="drug_approvals.csv"):
    # Open the CSV file for writing
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the CSV header
        writer.writerow(['Brand Name', 'Generic Name', 'Approval Date', 'Manufacturer', 'Drug Class'])
        
        page = 1
        total_results = 0

        # Keep fetching pages until there is no more data
        while True:
            print(f"Fetching page {page}...")
            results = fetch_drug_approvals(page)
            
            if not results:  # No more results, stop the loop
                print(f"All data fetched. Total records fetched: {total_results}")
                break

            # Process the results and write to CSV
            for result in results:
                brand_name = result.get('openfda', {}).get('brand_name', 'N/A')
                generic_name = result.get('openfda', {}).get('generic_name', ['N/A'])[0]
                approval_date = result.get('effective_time', 'N/A')
                manufacturer = result.get('openfda', {}).get('manufacturer_name', 'N/A')
                drug_class = result.get('openfda', {}).get('drug_class', ['N/A'])[0]
                
                # Write the data to the CSV file
                writer.writerow([brand_name, generic_name, approval_date, manufacturer, drug_class])

            total_results += len(results)
            page += 1
            
            # Delay between requests to avoid hitting rate limits
            time.sleep(DELAY_BETWEEN_REQUESTS)

# Run the function to fetch all drug approval data and save it to a CSV file
save_drug_approvals_to_csv('all_drug_approvals.csv')

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Error fetching data for page 27: 400
All data fetched. Total records fetched: 26000
